In [9]:
#Download the data
!wget https://biodynamics.ucsd.edu/wp-content/uploads/2020/08/features_targets_loadingrecord.zip
#Create a new folder for the data
!mkdir data_chem
#Unzip the data
!unzip features_targets_loadingrecord -d data_chem/
!rm features_targets_loadingrecord.zip
!echo "data_chem/" >> .gitignore

--2023-09-24 09:08:19--  https://biodynamics.ucsd.edu/wp-content/uploads/2020/08/features_targets_loadingrecord.zip
Resolving biodynamics.ucsd.edu (biodynamics.ucsd.edu)... 132.249.21.177
Connecting to biodynamics.ucsd.edu (biodynamics.ucsd.edu)|132.249.21.177|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 291529426 (278M) [application/zip]
Saving to: ‘features_targets_loadingrecord.zip’

features_targets_lo 100%[===================>] 278.02M  10.3MB/s    in 34s     

2023-09-24 09:08:54 (8.13 MB/s) - ‘features_targets_loadingrecord.zip’ saved [291529426/291529426]

Archive:  features_targets_loadingrecord.zip
  inflating: data_chem/features.csv  
  inflating: data_chem/loading_record.csv  
  inflating: data_chem/targets.csv   


In [193]:
import pandas as pd

In [194]:
#Load the matrix with all the data
data_full = pd.read_csv("./data_chem/features.csv")
#Load induction time profiles
u_ts = pd.read_csv("./data_chem/targets.csv")
#Load genes
genes = pd.read_csv("./data_chem/loading_record.csv")

In [195]:
#List and print all experiments
exp_id = data_full.exp_id.unique()
# print(exp_id)

#Select the experiment
exp_of_interest = 1 #this selects the first experiment only

#Subselecting data for the experiment of interest
df_exp = data_full[data_full.exp_id == exp_id[exp_of_interest-1]].drop(columns=["Unnamed: 1", "exp_id"])

#Renaming columns
df_exp.columns = genes.gene_name

#Filtering out spots with no strain averaging duplicate genes
df_exp=df_exp.iloc[:,df_exp.columns.notna()]

In [196]:
#In the original dataset there are several "biological replicates", i.e. spots of the chip where the same gene is tracked
#This snippet of code averages all these spots to obtain a final dataset with one column per gene
duplicated_columns = df_exp.columns[df_exp.columns.duplicated(keep=False)].unique()

for i in duplicated_columns:
    print("The gene ",i, " has ", df_exp[i].shape[1], " spots associated to it (biological replicates)")

# Create a new DataFrame to store the results
df_result = pd.DataFrame()

# Process non-duplicated columns
for column in df_exp.columns:
    if column not in duplicated_columns:
        df_result[column] = df_exp[column]

# Process duplicated columns
for dup_col in duplicated_columns:
    # Take the average of duplicated columns and assign to the result DataFrame
    df_result[dup_col] = df_exp[dup_col].mean(axis=1)

print(cnt)
df_result.head()

The gene  zntA  has  15  spots associated to it (biological replicates)
The gene  rpoS  has  15  spots associated to it (biological replicates)
The gene  cueO  has  15  spots associated to it (biological replicates)
The gene  rpoH  has  15  spots associated to it (biological replicates)
The gene  wrbA  has  21  spots associated to it (biological replicates)
The gene  rpoD  has  15  spots associated to it (biological replicates)
The gene  fimD  has  15  spots associated to it (biological replicates)
The gene  lacZ  has  22  spots associated to it (biological replicates)
The gene  yacH  has  2  spots associated to it (biological replicates)
The gene  U66  has  20  spots associated to it (biological replicates)
The gene  serA  has  21  spots associated to it (biological replicates)
The gene  U139  has  20  spots associated to it (biological replicates)
The gene  tdcG_2  has  2  spots associated to it (biological replicates)
The gene  infA  has  3  spots associated to it (biological replic

/tmp/ipykernel_1768/2366510112.py:14: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  df_result[column] = df_exp[column]


203


/tmp/ipykernel_1768/2366510112.py:19: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  df_result[dup_col] = df_exp[dup_col].mean(axis=1)


,rihC,ypfH,alsI,ygdQ,yabP,ypfJ,talB,yqeI,yabI,hda,...,rpoD,fimD,lacZ,yacH,U66,serA,U139,tdcG_2,infA,yebF
0,-0.022629,0.148508,-0.088955,-0.232066,0.069350,-0.644056,-0.030329,-0.020271,0.158288,1.246769,...,-0.084612,-0.012124,-0.064123,-0.065857,-0.063226,2.938553,-0.079553,-0.094431,-3.164757,-0.242952
1,-0.075618,-0.191642,-0.041127,0.221798,0.114038,-0.554063,-0.029043,0.117677,0.020580,1.160078,...,-0.051237,-0.009570,-0.067436,-0.047152,-0.096200,3.144924,-0.022218,-0.034728,-1.831284,0.094008
2,0.053473,0.146480,-0.111173,0.138315,0.002590,-0.147216,-0.113246,-0.030373,0.089998,1.312099,...,-0.020321,-0.031348,-0.067067,0.027820,-0.080628,3.385740,-0.017088,-0.034646,-3.035438,-0.126263
3,-0.137537,-0.224630,-0.055299,0.019529,0.085433,-0.508369,0.011168,-0.104683,0.235458,0.657336,...,-0.018129,-0.012545,-0.007957,-0.086683,-0.067020,2.658177,-0.048128,-0.101937,-2.332679,0.168084
4,-0.001089,0.149665,0.019627,0.292656,-0.153317,-0.116954,-0.076620,-0.048036,0.044441,1.461989,...,-0.072185,-0.016076,-0.097118,-0.055341,-0.110282,2.639522,-0.048160,-0.161916,-1.510021,0.566953


In [197]:
# #Saving THE experiment selected to a csv file
df_result.to_csv("./data_chem/exp"+str(exp_id[exp_of_interest-1])+".csv", index=False)